In [1]:

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter



# Read the clean_flavors_of_cacao.csv

In [2]:
# Load the data
file_path = Path('clean_flavors_of_cacao.csv')
df = pd.read_csv(file_path)
df.head()


,Company,Bean_Origin_or_Bar_Name,REF,Review_Date,Cocoa_Percent,Company_Location,Rating,Bean_Type,Broad_Bean_Origin
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru


In [3]:
df= df.drop(columns=["REF","Bean_Origin_or_Bar_Name"], axis=1)
df.head()

,Company,Review_Date,Cocoa_Percent,Company_Location,Rating,Bean_Type,Broad_Bean_Origin
0,A. Morin,2016,63.0,France,3.75,missing,Sao Tome & Principe
1,A. Morin,2015,70.0,France,2.75,missing,Togo
2,A. Morin,2015,70.0,France,3.00,missing,Togo
3,A. Morin,2015,70.0,France,3.50,missing,Togo
4,A. Morin,2015,70.0,France,3.50,missing,Peru


In [4]:
df['Rating'].mean()


3.18676552881925

In [5]:
df['Rating'].std()

0.47757934371049987

In [6]:
df['Rating'].median()

3.25

In [7]:
df['Rating'].describe()


count    1787.000000
mean        3.186766
std         0.477579
min         1.000000
25%         3.000000
50%         3.250000
75%         3.500000
max         5.000000
Name: Rating, dtype: float64

In [8]:
def bin_ratings(rating):
    if rating == 5.75: return 1
    if rating == 5.50: return 1
    if rating == 5.25: return 1
    if rating == 5.00: return 1
    
    if rating == 4.75: return 1
    if rating == 4.50: return 1
    if rating == 4.25: return 1
    if rating == 4.00: return 1
    
    if rating == 3.75: return 1
    if rating == 3.50: return 0
    if rating == 3.25: return 0
    if rating == 3.00: return 0

    if rating == 2.75: return 0
    if rating == 2.50: return 0
    if rating == 2.25: return 0
    if rating == 2.00: return 0
    
    if rating == 1.75: return 0
    if rating == 1.50: return 0
    if rating == 1.25: return 0
    if rating == 1.00: return 0
    
    #print( f"error: rating={rating} type={type(rating)}" )
    return "2"

In [9]:
df['Rating'] = df['Rating'].apply(bin_ratings)

# Split the Data into Training and Testing


In [10]:
y = df["Rating"]
X = df.drop(columns="Rating")
X = pd.get_dummies(X)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,test_size= 0.3,train_size=0.7 , random_state=1, stratify=y)

In [12]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Oversampling

In [13]:
#Naive Random Oversampling

# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
# Instantiate the model
ros = RandomOverSampler(random_state=1)
# Resample the targets
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)



Counter({0: 1035, 1: 1035})

In [14]:
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
# Calculate predictions
y_pred = model.predict(X_test)


C:\Users\Tata\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6794536471955828

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,336,108
Actual low_risk,37,56


In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.90      0.76      0.60      0.82      0.68      0.46       444
          1       0.34      0.60      0.76      0.44      0.68      0.45        93

avg / total       0.80      0.73      0.63      0.76      0.68      0.46       537



# Undersampling

In [18]:
# Resample the data using the ClusterCentroids resampler

from imblearn.under_sampling import ClusterCentroids
# Instantiate
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 215, 1: 215})

In [19]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

C:\Users\Tata\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.519180470793374

In [21]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,60,384
Actual low_risk,9,84


In [22]:


# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))



                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.14      0.90      0.23      0.35      0.11       444
          1       0.18      0.90      0.14      0.30      0.35      0.13        93

avg / total       0.75      0.27      0.77      0.25      0.35      0.12       537



# Combination (Over and Under) Sampling 

In [23]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 592, 1: 1382})

In [24]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

C:\Users\Tata\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.608435047951177

In [26]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,187,257
Actual low_risk,19,74


In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.42      0.80      0.58      0.58      0.32       444
          1       0.22      0.80      0.42      0.35      0.58      0.35        93

avg / total       0.79      0.49      0.73      0.54      0.58      0.33       537

